In [ ]:
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go

# Load data
data = pd.read_csv("RNDRUSDT_1.csv", parse_dates=['date'], index_col='date')

# Calculate VWAP
data.ta.vwap(append=True)

data

,open,high,low,close,volume,number_of_trades,VWAP_D
date,,,,,,,
2021-11-27,6.100,7.945,2.500,7.470,5385361.76,72851,5.971667
2021-11-28,7.470,7.827,6.083,6.948,2991578.89,33016,6.952667
2021-11-29,6.968,7.220,6.333,6.788,1362160.41,20534,6.780333
2021-11-30,6.795,6.824,6.138,6.268,1126567.79,16126,6.410000
2021-12-01,6.276,6.360,5.145,5.401,3006782.57,41587,5.635333
...,...,...,...,...,...,...,...
2024-05-24,10.234,10.342,9.750,10.110,5285923.03,196654,10.067333
2024-05-25,10.110,10.449,10.029,10.077,3818263.45,125499,10.185000
2024-05-26,10.079,10.179,9.864,9.988,3195301.24,109498,10.010333


In [ ]:
# Calculate Bollinger Bands
data.ta.bbands(length=10, std=1.5, append=True)

# Calculate SMA
data['SMA'] = ta.sma(data['close'], length=20)

# Calculate RSI
data['RSI'] = ta.rsi(data['close'], length=14)

data

,open,high,low,close,volume,number_of_trades,VWAP_D,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,SMA,RSI
date,,,,,,,,,,,,,,
2021-11-27,6.100,7.945,2.500,7.470,5385361.76,72851,5.971667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-28,7.470,7.827,6.083,6.948,2991578.89,33016,6.952667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-29,6.968,7.220,6.333,6.788,1362160.41,20534,6.780333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-30,6.795,6.824,6.138,6.268,1126567.79,16126,6.410000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-01,6.276,6.360,5.145,5.401,3006782.57,41587,5.635333,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-24,10.234,10.342,9.750,10.110,5285923.03,196654,10.067333,9.851331,10.4206,10.989869,10.925831,0.227194,10.45310,50.775978
2024-05-25,10.110,10.449,10.029,10.077,3818263.45,125499,10.185000,9.787512,10.3645,10.941488,11.133927,0.250861,10.46350,50.463176
2024-05-26,10.079,10.179,9.864,9.988,3195301.24,109498,10.010333,9.774412,10.3589,10.943388,11.284758,0.182714,10.46555,49.576132


In [ ]:
# Calculate MACD
macd = ta.macd(data['close'], fast=12, slow=26, signal=9)
data = pd.concat([data, macd], axis=1)

# Add the upper and lower bands to the DataFrame
data['Upper Band'] = data['BBU_10_1.5']
data['Lower Band'] = data['BBL_10_1.5']

# Drop NaN values
data.dropna(inplace=True)

In [ ]:
# Function to check trend based on moving average
def check_candles(data, backcandles, ma_column):
    categories = [0 for _ in range(backcandles)]
    for i in range(backcandles, len(data)):
        if all(data['close'][i-backcandles:i] > data[ma_column][i-backcandles:i]):
            categories.append(2)  # Uptrend
        elif all(data['close'][i-backcandles:i] < data[ma_column][i-backcandles:i]):
            categories.append(1)  # Downtrend
        else:
            categories.append(0)  # No trend
    return categories

In [ ]:
# Apply the function to the DataFrame
data['Trend'] = check_candles(data, 7, 'SMA')

# Entry Based on Bollinger Bands - Candles Crossing
data['entry'] = 0

# Condition for entry category 2 (buy entry)
buy_entry_condition = (data['Trend'] == 2) & ((data['open'] < data['Lower Band']) & (data['close'] > data['Lower Band']))
data.loc[buy_entry_condition, 'entry'] = 2

# Condition for entry category 1 (sell entry)
sell_entry_condition = (data['Trend'] == 1) & ((data['open'] > data['Upper Band']) & (data['close'] < data['Upper Band']))
data.loc[sell_entry_condition, 'entry'] = 1

# Filter entries
entries = data[data['entry'] != 0]

# Print entries
print(entries)

             open   high    low  close       volume  number_of_trades  \
date                                                                    
2022-04-20  2.144  2.207  2.024  2.053   2405742.23             18030   
2022-09-27  0.486  0.502  0.470  0.477   2336607.26              6740   
2023-02-10  1.375  1.525  1.318  1.483  40702255.62            177678   
2023-08-26  1.478  1.502  1.410  1.421   2776607.08             14138   

              VWAP_D  BBL_10_1.5  BBM_10_1.5  BBU_10_1.5  ...  BBP_10_1.5  \
date                                                      ...               
2022-04-20  2.094667    1.983251      2.0508    2.118349  ...    0.516284   
2022-09-27  0.483000    0.442692      0.4640    0.485308  ...    0.805043   
2023-02-10  1.442000    1.393864      1.6981    2.002336  ...    0.146491   
2023-08-26  1.444333    1.388292      1.4304    1.472508  ...    0.388382   

                SMA        RSI  MACD_12_26_9  MACDh_12_26_9  MACDs_12_26_9  \
date                

In [ ]:
# Plotting the data
fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'],
                name='Candlesticks')])

# Add the moving averages and other indicators to the plot
fig.add_trace(go.Scatter(x=data.index, y=data['SMA'], mode='lines', name='SMA', line=dict(color='red')))
fig.add_trace(go.Scatter(x=data.index, y=data['Lower Band'], mode='lines', name='Lower Band', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=data.index, y=data['Upper Band'], mode='lines', name='Upper Band', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=data.index, y=data['VWAP_D'], mode='lines', name='VWAP_D', line=dict(color='purple')))
fig.add_trace(go.Scatter(x=data.index, y=data['RSI'], mode='lines', name='RSI', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=data.index, y=data['MACD_12_26_9'], mode='lines', name='MACD', line=dict(color='green')))
fig.add_trace(go.Scatter(x=data.index, y=data['MACDh_12_26_9'], mode='lines', name='MACD Histogram', line=dict(color='cyan')))
fig.add_trace(go.Scatter(x=data.index, y=data['MACDs_12_26_9'], mode='lines', name='MACD Signal', line=dict(color='magenta')))

fig.update_layout(
    title='Technical Analysis on RNDRUSDT',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    template='plotly_dark'
)

fig.show()